# Часть 3: Интеграция с MLflow

## 1. MLFlow server tracking

mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts --host 0.0.0.0 --port 5000

## 2. Полный жизненный цикл модели

In [22]:
from feast import FeatureStore
import pandas as pd
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import mlflow
from mlflow.tracking import MlflowClient
import uuid
import time
import os

In [23]:
mlflow.set_tracking_uri("http://localhost:5000")

EXPERIMENT_NAME = "Taxi Demand Prediction"
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='/home/nockerox/Dev/bigdata/lab4/1/artifacts/2', creation_time=1762958352550, experiment_id='2', last_update_time=1762958352550, lifecycle_stage='active', name='Taxi Demand Prediction', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [24]:
store = FeatureStore(repo_path="../2/nyc_taxi_demand/feature_repo")

In [25]:
df_features = pd.read_parquet('../2/nyc_taxi_demand/feature_repo/data/demand_agg_with_ts.parquet')
df_features.head()

,PULocationID,pickup_hour,trip_count,hour,dayofweek,lag_1h,lag_24h,lag_168h,rolling_mean_24h
0,1,2019-02-01 01:00:00,1,1,4,0.0,0.0,0.0,0.0
1,1,2019-02-01 06:00:00,2,6,4,1.0,0.0,0.0,0.0
2,1,2019-02-01 07:00:00,1,7,4,2.0,0.0,0.0,0.0
3,1,2019-02-01 08:00:00,3,8,4,1.0,0.0,0.0,0.0
4,1,2019-02-01 09:00:00,1,9,4,3.0,0.0,0.0,0.0


In [26]:
min_date = df_features['pickup_hour'].min()
max_date = df_features['pickup_hour'].max()
date_definition = min_date + (max_date - min_date) * (1 - 0.8)
print(f"Дата разделения: {date_definition}")
print(f"Начало данных: {min_date}, конец данных: {max_date}")

Дата разделения: 2019-02-06 14:11:59.999999
Начало данных: 2019-02-01 00:00:00, конец данных: 2019-02-28 23:00:00


In [27]:
date_middle = min_date + (date_definition - min_date) * (1 - 0.5)
print(f"Дата середины: {date_middle}")

Дата середины: 2019-02-03 19:05:59.999999


In [28]:
date_for_train = min_date + (date_definition - min_date) * (1 - 0.5) * 0.7
print(f"Дата для обучения: {date_for_train}")

Дата для обучения: 2019-02-02 22:58:11.999999


In [29]:
train_entity_df = df_features[df_features['pickup_hour'] <= date_for_train].copy()
train_entity_df = train_entity_df[['pickup_hour', 'PULocationID']]
train_entity_df['PULocationID'] = train_entity_df['PULocationID'].astype('int64')

print(f"\nПолучаем исторические фичи для {len(train_entity_df)} событий...")

features_to_get = [
    "taxi_stats:trip_count",
    "taxi_stats:lag_1h",
    "taxi_stats:lag_24h",
    "taxi_stats:lag_168h",
    "taxi_stats:rolling_mean_24h",
    "taxi_stats:hour",
    "taxi_stats:dayofweek",
]

train_df = store.get_historical_features(
    entity_df=train_entity_df,
    features=features_to_get,
).to_df()

print("Обучающий датасет из Feast:")
print(train_df.head())


Получаем исторические фичи для 11869 событий...
Using pickup_hour as the event timestamp. To specify a column explicitly, please name it event_timestamp.
Обучающий датасет из Feast:
                pickup_hour  PULocationID  trip_count  lag_1h  lag_24h  \
0 2019-02-01 00:00:00+00:00           246         163     0.0      0.0   
1 2019-02-01 00:00:00+00:00            14          71     0.0      0.0   
2 2019-02-01 00:00:00+00:00            37         361     0.0      0.0   
3 2019-02-01 00:00:00+00:00            86          18     0.0      0.0   
4 2019-02-01 00:00:00+00:00           208          31     0.0      0.0   

   lag_168h  rolling_mean_24h  hour  dayofweek  
0       0.0               0.0     0          4  
1       0.0               0.0     0          4  
2       0.0               0.0     0          4  
3       0.0               0.0     0          4  
4       0.0               0.0     0          4  


In [30]:
test_entity_df = df_features[(df_features['pickup_hour'] > date_for_train) & (df_features['pickup_hour'] <= date_middle)].copy()
test_entity_df = test_entity_df[['pickup_hour', 'PULocationID']]
test_entity_df['PULocationID'] = test_entity_df['PULocationID'].astype('int64')
print(f"\nПолучаем исторические фичи для {len(test_entity_df)} событий...")
test_df = store.get_historical_features(
    entity_df=test_entity_df,
    features=features_to_get,
).to_df()
print("Тестовый датасет из Feast:")
print(test_df.head())


Получаем исторические фичи для 5299 событий...
Using pickup_hour as the event timestamp. To specify a column explicitly, please name it event_timestamp.
Тестовый датасет из Feast:
                pickup_hour  PULocationID  trip_count  lag_1h  lag_24h  \
0 2019-02-02 23:00:00+00:00            76         379   361.0    297.0   
1 2019-02-02 23:00:00+00:00             8           1     2.0      0.0   
2 2019-02-02 23:00:00+00:00           258         117   121.0     85.0   
3 2019-02-02 23:00:00+00:00           226         279   215.0    226.0   
4 2019-02-02 23:00:00+00:00           219          56    57.0     30.0   

   lag_168h  rolling_mean_24h  hour  dayofweek  
0       0.0        277.000000    23          5  
1       0.0          0.000000    23          5  
2       0.0         91.250000    23          5  
3       0.0        209.000000    23          5  
4       0.0         39.583333    23          5  


In [31]:
lgbm_params = {
    'random_state': 42,
    'n_estimators': 100,
    'learning_rate': 0.1,
    'num_leaves': 31
}


with mlflow.start_run(run_name='LGBM_Pipeline_Baseline_v3'):
    mlflow.log_params(lgbm_params)
    regressor = lgb.LGBMRegressor(random_state=42)

    model_pipeline = Pipeline([
        ('regressor', regressor)
    ])

    model_pipeline.set_params(regressor__random_state=lgbm_params['random_state'],
                           regressor__n_estimators=lgbm_params['n_estimators'],
                           regressor__learning_rate=lgbm_params['learning_rate'],
                           regressor__num_leaves=lgbm_params['num_leaves'])

    print("--- Обучение пайплайна ---")

    X_train = train_df.drop(columns=['pickup_hour', 'PULocationID', 'trip_count'])
    y_train = train_df['trip_count']
    X_test = test_df.drop(columns=['pickup_hour', 'PULocationID', 'trip_count'])
    y_test = test_df['trip_count']

    model_pipeline.fit(X_train, y_train)
    print("Готово.")

    print("\n--- Оценка модели ---")
    predictions = model_pipeline.predict(X_test)

    mae = mean_absolute_error(y_test, predictions)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    r2 = r2_score(y_test, predictions)

    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R2 Score: {r2:.4f}")

    mlflow.log_metrics({"mae": mae, "rmse": rmse, "r2_score": r2})

    input_example = X_train.head(5)

    mlflow.sklearn.log_model(
        sk_model=model_pipeline,
        name="demand_forecasting_pipeline",
        registered_model_name="LGBM-Demand-Forecaster",
        input_example=input_example
    )

--- Обучение пайплайна ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000309 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 792
[LightGBM] [Info] Number of data points in the train set: 11869, number of used features: 5
[LightGBM] [Info] Start training from score 140.817508
Готово.

--- Оценка модели ---
MAE: 21.36
RMSE: 47.34
R2 Score: 0.9047


/home/nockerox/Dev/bigdata/.venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/11/13 20:47:06 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'LGBM-Demand-Forecaster' already

🏃 View run LGBM_Pipeline_Baseline_v3 at: http://localhost:5000/#/experiments/2/runs/114a64530c344cfb85a5ad6a46c720b6
🧪 View experiment at: http://localhost:5000/#/experiments/2


Created version '18' of model 'LGBM-Demand-Forecaster'.


## 3-4. Сравнение моделей и валидация

In [32]:
current_experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
experiment_id = current_experiment.experiment_id

df_runs = mlflow.search_runs([experiment_id], order_by=["metrics.r2_score DESC"])
best_run_id = df_runs.loc[0, 'run_id']

model_uri = "runs:/" + best_run_id + "/" + "LGBM-Demand-Forecaster"
print(model_uri)

runs:/114a64530c344cfb85a5ad6a46c720b6/LGBM-Demand-Forecaster


In [33]:
client = MlflowClient()
client.transition_model_version_stage(
    name='LGBM-Demand-Forecaster',
    version=7,
    stage='Production'
)

model = mlflow.pyfunc.load_model("models:/LGBM-Demand-Forecaster/Production")

X_test = test_df.drop(columns=['pickup_hour', 'PULocationID', 'trip_count'])
y_test = test_df['trip_count']

predictions = model.predict(X_test)

mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2 Score: {r2:.4f}")

/tmp/ipykernel_8573/1022956840.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


MAE: 21.36
RMSE: 47.34
R2 Score: 0.9047


In [34]:
def validate_model(model, x_test, y_test):
    predictions = model.predict(x_test)
    r2 = r2_score(y_test, predictions)
    if r2 < 0.85:
        return False
    return True

validate_model(model, X_test, y_test)

True

In [35]:
X_train = train_df.drop(columns=['pickup_hour', 'PULocationID', 'trip_count'])
y_train = train_df['trip_count']

X_test = test_df.drop(columns=['pickup_hour', 'PULocationID', 'trip_count'])
y_test = test_df['trip_count']

reference_data_full = pd.concat([X_train, y_train], axis=1)
current_data_full = pd.concat([X_test, y_test], axis=1)

os.makedirs('monitoring/data', exist_ok=True)

reference_data_full.to_parquet('monitoring/data/reference_data.parquet')
current_data_full.to_parquet('monitoring/data/current_data.parquet')

print("Данные для мониторинга сохранены.")

Данные для мониторинга сохранены.
